In [ ]:
# If data not stored locally run this cell to fetch it
"""

import kagglehub
import os

# Define the download directory
download_dir = os.path.join("data_full", "raw")

# Create directory if it doesn't exist
os.makedirs(download_dir, exist_ok=True)

# Download dataset to specified directory
path = kagglehub.dataset_download("steubk/wikiart", path=download_dir)

print("Path to dataset files:", path)

"""

In [ ]:
import os
from PIL import Image
import torchvision.transforms as transforms
from torchvision.utils import save_image

# Define paths
input_dir = "data\preprocessed_images"  # Folder containing raw artwork images
output_dir = "data_sample\preprocessed_images"  # Output folder for preprocessed images

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Define transformations
preprocess = transforms.Compose([
    transforms.Resize(256),          # Resize shorter side to 256
    transforms.CenterCrop(224),      # Crop center to 224x224
    transforms.ToTensor(),           # Convert to tensor [0, 255] → [0, 1]
    # Choose ONE normalization method below:
    
    # Method 1: Scale to [0, 1] (divide by 255)
    # (No additional step needed; ToTensor() already scales to [0, 1])
    
    # Method 2: ImageNet Normalization (mean/std)
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Process all images
for root, _, files in os.walk(input_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(root, file)
            try:
                # Load image
                img = Image.open(img_path).convert("RGB")  # Ensure RGB format
                
                # Apply preprocessing
                img_tensor = preprocess(img)
                
                # Save preprocessed image
                output_path = os.path.join(output_dir, file)
                save_image(img_tensor, output_path)
                
            except Exception as e:
                print(f"Error processing {img_path}: {e}")

In [ ]:
# Load a preprocessed image to check values
test_img = Image.open(os.path.join(output_dir, "sample_image.jpg"))
test_tensor = transforms.ToTensor()(test_img)

print("Tensor shape:", test_tensor.shape)  # Should be (3, 224, 224)
print("Min/Max pixel value:", test_tensor.min(), test_tensor.max())  # Should be ~[0, 1] or [-2.1, 2.5] for ImageNet

In [ ]:
# Convert images to embeddings.
import torch
from torchvision import models, transforms

# Load pre-trained ResNet-50 and remove the last layer
model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*(list(model.children())[:-1]))
model.eval()

# Example transformation pipeline
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Extract embeddings for an image
image = transform(Image.open("path/to/image.jpg")).unsqueeze(0)
embedding = model(image).squeeze().detach().numpy()

In [ ]:
# EDA
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Reduce embeddings to 2D for visualization
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings)

plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.5)
plt.title("PCA of Artwork Embeddings")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.show()

In [ ]:
# Dimensionality Reduction
import umap
reducer = umap.UMAP(n_components=2, random_state=42)
embeddings_umap = reducer.fit_transform(embeddings)